**Ultimate script to prepare the raw freesurfer data**
------------------------------------------------------

- just put everything into one textfile
- check
- add clinics

*cross sectional AND longitudinal*

**Dependencies**

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import os
from glob import glob

# importing custom functions
code_dir = ('/home/barbora/Documents/Projects/Normative_Models/ESO/code')
os.chdir(code_dir)
import clinics_desc_functions as custom


In [2]:
main_dir = '/home/barbora/Documents/Projects/Normative_Models' 
cs_dir = '/home/barbora/Documents/Projects/Normative_Models/ESO/backup/fit_external_cs'
long_dir = '/home/barbora/Documents/Projects/Normative_Models/ESO/backup/fit_external_long'
target_dir = '/home/barbora/Documents/Projects/Normative_Models/ESO/models/sensitivity/data'

**Clinics**

In [3]:
# Import clinics
clinics = pd.read_excel(os.path.join(main_dir, 'ESO', 'clinics_parsed.xlsx'), thousands=',')

In [4]:
# Creating and adding index
index = [i.lstrip('ESO') for i in clinics['osobní kód (Hydra ID).1']]
clinics.index = index

In [5]:
# Rename columns and categorical variables
clinics['category'] = clinics['Kategorie osoby'].str[:].str.upper().map({'PACIENT':'Patient', 'KONTROLA':'Control', 'SOUROZENEC':'Sibling', 'HIGH RISK':'High_risk'})
clinics['inclusion'] = clinics['Zařazení osoby'].str[:].str.upper().map({'ZAŘAZENA':'Included', 'ZAŘAZENA S VÝHRADAMI':'Included with Reservations'})

sex_nums = {"Pohlaví": {"žena":0, "muž":1}}
clinics = clinics.replace(sex_nums)

clinics.rename(columns={"Pohlaví":"sex", 
                        "Je pravák?":"right_handed", 
                        "Věk při 1. vizitě":"age_1", 
                        "Věk při 2. vizitě":"age_2", 
                        "Věk při 3. vizitě":"age_3",
                        "Klinika/DUP [měsíce] 1":"length_of_symptoms",
                        "Klinika/Délka onem. do MRI 1 [měsíce] 1":"length_of_disease",
                        "Klinika/Dg. 1":"diagnosis",},
                        inplace=True)

for i in range(1,4):
    clinics['age_'+str(i)] = clinics['age_'+str(i)]/1000

**Pick Preprocessing**


In [76]:
# Load
preproc = 'long' # cs or long
modality = 'SurfArea' #'SurfArea' or 'thickness'

if preproc == 'long':
    v1 = pd.read_csv(glob(long_dir+'/*'+modality+'*_1.txt')[0], sep =';', index_col=0)
    v2 = pd.read_csv(glob(long_dir+'/*'+modality+'*_2.txt')[0], sep =';', index_col=0)
if preproc == 'cs':
    v1_c = pd.read_csv(glob(cs_dir+'/*_C_'+modality+'*_1.csv')[0], sep =';', index_col=0)
    v2_c = pd.read_csv(glob(cs_dir+'/*_C_'+modality+'*_2.csv')[0], sep =';', index_col=0)
    v1_p = pd.read_csv(glob(cs_dir+'/*_P_'+modality+'*_1.csv')[0], sep =';', index_col=0)
    v2_p = pd.read_csv(glob(cs_dir+'/*_P_'+modality+'*_2.csv')[0], sep =';', index_col=0)

    v1 = pd.concat([v1_c, v1_p])
    v2 = pd.concat([v2_c, v2_p])


In [77]:
# rename indicies for future merging
index = [i.split('_')[0] for i in v1.index]
v1.index = index

index = [i.split('_')[0] for i in v2.index]
v2.index = index

# Delete siblings and high risk data
v1.drop(v1.iloc[np.where(np.array([i[0] for i in v1.index]) == 'S')[0]].index, inplace = True)
v2.drop(v2.iloc[np.where(np.array([i[0] for i in v2.index]) == 'S')[0]].index, inplace = True)

v1.drop(v1.iloc[np.where(np.array([i[0] for i in v1.index]) == 'H')[0]].index, inplace = True)
v2.drop(v2.iloc[np.where(np.array([i[0] for i in v2.index]) == 'H')[0]].index, inplace = True)


# Do we need to delete some other data? 
if (v1.index.intersection(v2.index).shape[0] == v1.shape[0]) & (v1.index.intersection(v2.index).shape[0] == v2.shape[0]):
    print('All good dataset are same sizes with same subjects')
else: 
    print('Something is not good, some subjects might be redundant, chceck teh intersection of visits')

# Sort and get ready for possible concatenation
v1 = v1.sort_index()
v2 = v2.sort_index()

sum(v1.index == v2.index) == v1.shape[0]

All good dataset are same sizes with same subjects


True

*Adding the clinic*

In [78]:
visits = [v1, v2]

# visit 1 
for i, visit in enumerate(visits):
    
    # Basics
    visit = pd.concat([clinics[['category', 'sex', 'age_'+str(i+1), "diagnosis", "length_of_symptoms", "length_of_disease"]],visit], axis=1, join='inner')
    visit['visit'] = i+1
    visit['preproc'] = preproc
    visit['site'] = 'NUDZ'
    visit['sitenum'] = 1000
    visit.rename(columns={"age_"+str(i+1):"age"}, inplace=True)
    #visit['diagnosis'] = visit['diagnosis'].astype('category')
    ###
    # PANSS
    ###

    PANSS = [col for col in clinics.columns if "PANSS "+str(i+1) in col]

    visit = pd.concat([visit, clinics[PANSS]], join="inner", axis=1)
    visit.rename(columns=lambda x: x.replace('PANSS/PANSS '+str(i+1)+' / ', 'PANSS_'), inplace=True)

    # Rename the PANSS variables
    PANSS = [col for col in visit.columns if "PANSS_" in col]

    # We need to replace empy values (originally described as 'x') with NaN
    mapping = {'x': np.nan}
    visit = visit.applymap(lambda s: mapping.get(s) if s in mapping else s)

    # Create indicies of the three PANSS chapters
    PANSS_P = [col for col in visit.columns if "PANSS_P" in col][0:-1]
    PANSS_N = [col for col in visit.columns if "PANSS_N" in col][0:-1]
    PANSS_G = [col for col in visit.columns if "PANSS_G" in col][0:-1]

    visit["PANSS_sumP"] = visit[[col for col in visit.columns if "PANSS_P" in col][0:-1]].sum(axis=1)
    visit["PANSS_sumN"] = visit[[col for col in visit.columns if "PANSS_N" in col][0:-1]].sum(axis=1)
    visit["PANSS_sumG"] = visit[[col for col in visit.columns if "PANSS_G" in col][0:-1]].sum(axis=1)


    ###
    # GAF
    ###
    GAF = [col for col in clinics.columns if "Škály/GAF "+str(i+1) in col]

    visit = pd.concat([visit, clinics[GAF]], join="inner", axis=1)
    visit.rename(columns={GAF[0]:"GAF"}, inplace=True)

    visit['full_index']=visit.index+'_'+str(i+1)

    ###
    # REMOVING SUBJECTS BASED ON INCLUSION/EXCLUSION CRITERIA
    ###
    # dele subjects younger than 18
    visit = visit.loc[visit['age']>=18]
    # delete subjects with disease duration longer than 24 months
    visit = visit.loc[(visit['length_of_disease']<=24) | (visit['category']=='Control')]
    # dele subjects with diagnosis F25
    visit = visit.loc[visit['diagnosis'].isnull() | visit['diagnosis'].str.contains('F20|F23')]

    # change data in the list
    visits[i] = visit


**Creating one more dataset that is a m erge of both visits (if needed for plottng or something)**

In [79]:
all = pd.concat([visits[0], visits[1]], axis=0)
all = all.reset_index()
all.index = all['full_index']

**Save**

In [80]:
visits[0].to_csv(os.path.join(target_dir, 'all_'+preproc+'_'+modality+'_1.csv'), sep=' ', index=True)
visits[1].to_csv(os.path.join(target_dir, 'all_'+preproc+'_'+modality+'_2.csv'), sep=' ', index=True)
all.to_csv(os.path.join(target_dir, 'all_'+preproc+'_'+modality+'.csv'), sep=' ', index=True)

**Euler number is not available for longitudinal data, so we are inserting the cs one**

In [81]:
all_long = pd.read_csv(os.path.join(target_dir, 'all_long_'+modality+'.csv'), sep=' ', index_col=0)
all_cs = pd.read_csv(os.path.join(target_dir, 'all_cs_'+modality+'.csv'), sep=' ', index_col=0)

if all_long.shape[0] == all_cs.shape[0]:
    print('same number of subjects in long and cs dataset')

all_long = pd.concat([all_long, all_cs[all_cs.columns.difference(all_long.columns)]], axis = 1, join='inner')
all_long.shape[0] == all_cs.shape[0]

all_long.to_csv(os.path.join(target_dir, 'all_long_'+modality+'.csv'), sep=' ', index=True)

same number of subjects in long and cs dataset


In [82]:
for i in range(1,3):

    all_long = pd.read_csv(os.path.join(target_dir, 'all_long_'+modality+'_'+str(i)+'.csv'), sep=' ', index_col=0)
    all_cs = pd.read_csv(os.path.join(target_dir, 'all_cs_'+modality+'_'+str(i)+'.csv'), sep=' ', index_col=0)

    if all_long.shape[0] == all_cs.shape[0]:
        print('same number of subjects in long and cs dataset')

    all_long = pd.concat([all_long, all_cs[all_cs.columns.difference(all_long.columns)]], axis = 1, join='inner')
    all_long.shape[0] == all_cs.shape[0]

    all_long.to_csv(os.path.join(target_dir, 'all_long_'+modality+'_'+str(i)+'.csv'), sep=' ', index=True)


same number of subjects in long and cs dataset
same number of subjects in long and cs dataset


**QC**
---------

In [83]:
v1 = pd.read_csv(os.path.join(target_dir, 'all_'+preproc+'_'+modality+'_1.csv'), sep=' ', index_col=0)
v2 = pd.read_csv(os.path.join(target_dir, 'all_'+preproc+'_'+modality+'_2.csv'), sep=' ', index_col=0)
all = pd.read_csv(os.path.join(target_dir, 'all_'+preproc+'_'+modality+'.csv'), sep=' ', index_col=0)

In [74]:
v1.drop(columns = 'SurfaceHoles', inplace = True)
v2.drop(columns = 'SurfaceHoles', inplace = True)

v1 = custom.en_qc(v1)
v2 = custom.en_qc(v2)

v1 = v1.loc[v1.index.intersection(v2.index)]
v2 = v2.loc[v1.index.intersection(v2.index)]

all = all.iloc[np.where(all['index'].isin(list(v1.index.intersection(v2.index)))==True)[0]]

In [75]:
all.to_csv(os.path.join(target_dir, 'all_'+preproc+'_'+modality+'_qc.csv'), sep=' ', index=True)
v1.to_csv(os.path.join(target_dir, 'all_'+preproc+'_'+modality+'_1_qc.csv'), sep=' ', index=True)
v2.to_csv(os.path.join(target_dir, 'all_'+preproc+'_'+modality+'_2_qc.csv'), sep=' ', index=True)